In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
fp = "data/gridtx-dump.csv"
dtype={'adjusts_id': 'object',
    'ediel_product_code': 'float64',
    'invoice_item_id': 'object',
    'parent_id': 'object'}

data = dd.read_csv(fp, sep=";", dtype=dtype)

dtype={'meter_number': 'object'}
meters = dd.read_csv("data/grid-metering-point-dump-plus-zipcode-GMAPS.csv", dtype=dtype)

def list_diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

to_keep_meters = ["idgrid_data_table", "market_balance_area", "automatic_meter", "lat", "lng"]
to_drop_meters = list_diff(to_keep_meters, list(meters.columns))

to_keep_data = ['created_at', 'updated_at', 'num_kwh', 'timeslot', 'type', 'estimation', 'spot_price_no_vat', 'amount_no_vat', 'vat_percent', 'meter_id', 'kwh_fee_no_vat']
to_drop_data = list_diff(to_keep_meters, list(data.columns))

data_merged = dd.merge(data, meters.drop(to_drop_meters, axis = 1), how="left", left_on="meter_id", right_on = "idgrid_data_table")

data_merged.to_csv("data/gridtx-dump-MERGED.csv", index= None, header = True,
    single_file=True)